In [0]:
import helper
import numpy as np
import torch
import torch.nn as nn
from torch_utils import batch_data, train_rnn
from models import RNN
import time
import os
import os
import pickle
# hyperparameters
# 3, 5 ,7
sequence_length = 3
batch_size = 128
num_epochs = 10
learning_rate = 0.002
embedding_dim = 256
hidden_dim = 256
n_layers = 2
show_every_n_batches = 500

In [0]:
import os
import importlib
importlib.reload(helper)

<module 'helper' from '/content/helper.py'>

In [0]:
# int_text, vocab_to_int, int_to_vocab = helper.load_preprocess()
int_text, vocab_to_int, int_to_vocab = pickle.load(open('preprocess_wv.pkl', mode='rb'))
train_loader = batch_data(int_text, sequence_length, batch_size)
vocab_size = len(vocab_to_int)
output_size = len(vocab_to_int)

In [0]:
# rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.25)
# if train_on_gpu:
#     rnn.cuda()

In [0]:
import torch
import helper
import torch.nn.functional as F
import time

def generate(rnn, prime_id, int_to_vocab, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    train_on_gpu = torch.cuda.is_available()

    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]

    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)

        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))

        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)

        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu

        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()

        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())

        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)
        current_seq = current_seq.cpu().data.numpy()
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i

    listOfPeople = ['howard','sheldon', 'penny', 'raj', 'bernadette', 'amy', 'leonard']
    for i in range(1, len(predicted)):
      if predicted[i] in listOfPeople:
        predicted[i-1] = predicted[i]+"."
    gen_sentences = ' '.join(predicted)

    # Replace punctuation tokens
    # for key, token in token_dict.items():
    #     ending = ' ' if key in ['\n', '(', '"'] else ''
    #     gen_sentences = gen_sentences.replace(' ' + token, key)
    # gen_sentences = gen_sentences.replace('\n ', '\n')
    # gen_sentences = gen_sentences.replace('( ', '(')

    # return all the sentences
    return gen_sentences

In [0]:
_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

FileNotFoundError: ignored

In [0]:
from nltk.translate.bleu_score import sentence_bleu
import random
def selfBleu(generated_script):
    removeScene = generated_script.index(".")
    dialogue = generated_script[removeScene+1:]
    removeNames = []
    splitByWords = dialogue.split(" ")
    for i in range(0, len(splitByWords)-1):
      if ":" not in splitByWords[i]:
        removeNames.append(splitByWords[i])
    # print(removeNames)
    sentences = " ".join(removeNames).split(".")
    wordSentences = []
    for i in sentences:
      tempList = i.split(" ")
      wordSentences.append(tempList[1:])
    # print()
    # print(wordSentences)

    # generated_script
    totalScore = 0.0
    for i in wordSentences:
      temp = list(wordSentences)
      temp.remove(i)
      # print(temp)
      # print(i)
      score = sentence_bleu(temp, i)
      totalScore += score
    returnScore = totalScore/len(wordSentences)
    print(returnScore)

In [0]:
def bleu(generated_script):
    tempDict = helper.load_data("./data.pkl")
    keyList = list(tempDict.keys())
    episodeList = []
    removeScene = generated_script.index(".")
    dialogue = generated_script[removeScene+1:]
    removeNames = []
    splitByWords = dialogue.split(" ")
    for i in range(0, len(splitByWords)-1):
      if ":" not in splitByWords[i]:
        removeNames.append(splitByWords[i])
    # print(removeNames)
    sentences = " ".join(removeNames).split(".")
    for h in keyList[:10]:
      finalList = []
      for i in tempDict[h]:
        if "scene:" not in i.lower():
          indexColon = i.index(":")
          tempDialogue = i.lower()[indexColon+1:]
          if "(" in tempDialogue:
            tempDialogue = tempDialogue[:tempDialogue.index("(")]
          finalList.append(tempDialogue)
      sentencesFinal = []
      for j in finalList:
        sentencesFinal.extend(j.split("."))
      wordOFSentence = []
      for i in sentencesFinal:
        wordOFSentence.append(i.split(" ")[1:])
      episodeList.extend(wordOFSentence)
    wordSentences = []
    for i in sentences:
      tempList = i.split(" ")
      wordSentences.append(tempList[1:])
    totalScore = 0.0
    for i in wordSentences:
      score = sentence_bleu(episodeList, i)
      totalScore += score
    returnScore = totalScore/len(wordSentences)
    print(returnScore)

In [0]:
class Word2VecModel(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, syn0, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(Word2VecModel, self).__init__()

        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.window_length = 140
        self.syn0 = syn0

        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)

        # dropout layer
        self.dropout = nn.Dropout(dropout)

        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        # self.sig = nn.Softmax(dim=1)
        self.init_weights()


    def init_weights(self):
        initrange = 0.08
        self.embedding.weight.data.copy_(torch.from_numpy(self.syn0))
        self.embedding.weight.requires_grad = False
        # self.embedding.weight.data.uniform_(-initrange, initrange)
        self.lstm.weight_ih_l0.data.uniform_(-initrange, initrange)
        self.lstm.weight_hh_l0.data.uniform_(-initrange, initrange)

        self.lstm.bias_ih_l0.data.zero_()
        self.lstm.bias_hh_l0.data.zero_()

        # self.fc.bias.data.zero_()
        self.fc.bias.data.fill_(0)
        # self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.normal_(0.0, (1.0 / np.sqrt(self.fc.in_features)))


    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """

        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.dropout(self.embedding(nn_input))
        # lstm_out, hidden = self.lstm(embeds.view(len(nn_input), self.window_length, -1), hidden)

        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs

        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        sig_out = self.fc(lstm_out)
        # out = self.fc1(out)
        # print(sig_out.shape)

        sig_out = sig_out.view(batch_size, -1, self.output_size)
        sig_out = sig_out[:, -1]

        # sig_out = F.log_softmax(sig_out, dim=1)

        # return one batch of output word scores and the hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''

        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data

        if (torch.cuda.is_available()):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden

In [0]:
from torch.autograd import Variable
class W2VVanilla(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, syn0, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(W2VVanilla, self).__init__()

        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.window_length = 140
        self.syn0 = syn0

        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(self.syn0))
        self.embedding.weight.requires_grad = False
        self.lstm = nn.RNN(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)

        # dropout layer
        self.dropout = nn.Dropout(dropout)

        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        # self.sig = nn.Softmax(dim=1)


    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """

        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.dropout(self.embedding(nn_input))
        # lstm_out, hidden = self.lstm(embeds.view(len(nn_input), self.window_length, -1), hidden)

        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs

        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        sig_out = self.fc(lstm_out)
        # out = self.fc1(out)
        # print(sig_out.shape)

        sig_out = sig_out.view(batch_size, -1, self.output_size)
        sig_out = sig_out[:, -1]

        # sig_out = F.log_softmax(sig_out, dim=1)

        # return one batch of output word scores and the hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        if (torch.cuda.is_available()):
            hidden = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim).cuda())
        else:
            hidden = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim))

        return hidden

In [0]:
class W2VGRU(nn.Module):

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, syn0, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(W2VGRU, self).__init__()

        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.window_length = 140
        self.syn0 = syn0

        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(self.syn0))
        self.lstm = nn.GRU(embedding_dim, hidden_dim, n_layers, dropout=dropout, batch_first=True)

        # dropout layer
        self.dropout = nn.Dropout(dropout)

        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        # self.sig = nn.Softmax(dim=1)


    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """

        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        embeds = self.dropout(self.embedding(nn_input))
        # lstm_out, hidden = self.lstm(embeds.view(len(nn_input), self.window_length, -1), hidden)

        lstm_out, hidden = self.lstm(embeds, hidden)

        # stack up lstm outputs

        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

        sig_out = self.fc(lstm_out)
        # out = self.fc1(out)
        # print(sig_out.shape)

        sig_out = sig_out.view(batch_size, -1, self.output_size)
        sig_out = sig_out[:, -1]

        # sig_out = F.log_softmax(sig_out, dim=1)

        # return one batch of output word scores and the hidden state
        return sig_out, hidden


    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        if (torch.cuda.is_available()):
            hidden = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim).cuda())
        else:
            hidden = Variable(torch.zeros(self.n_layers, batch_size, self.hidden_dim))

        return hidden


In [0]:
import datetime
# if you are running in uploaded model
from os import listdir
from os.path import isfile, join
onlyfiles = ['trained_w2v_gru', 'trained_w2v_vanilla', 'trained_lstm']
print(onlyfiles)
gen_length = 700
prime_word = 'scene' # name for starting the script
pad_word = helper.SPECIAL_WORDS['PADDING']

for i in onlyfiles:
  trained_rnn = torch.load('./models/'+i+'.pt')
  generated_script = generate(trained_rnn, vocab_to_int[prime_word + ''], int_to_vocab, 10, gen_length)
  print(i)
  print(generated_script)
  bleu(generated_script=generated_script)
  selfBleu(generated_script=generated_script)
# generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100)
# print(generated_script)


# f =  open("generated_script" + str(datetime.datetime.now()) + ".txt","w")
# f.write(generated_script)
# f.close()

['trained_w2v_gru', 'trained_w2v_vanilla', 'trained_lstm']
trained_w2v_gru
howard. howard leonard. leonard hey ames williams leonard. leonard hey guys are not going to be prepared to do you want to go out with raj. raj so if you re leonard. leonard you re kidding you re howard. howard what do it provided leonard. leonard yeah well you don have to go out of your penny. penny what do you want to go out of the handbook of the leonard. leonard what leonard. leonard what you re talking about sheldon. sheldon yeah well what you do you want you to meet me hopefully you re both sheldon. sheldon no no no no no it was like it to say that you re going to talk about leonard. leonard oh you know what do you want to make me blink to go to sheldon. sheldon oh well we re not going to be staying at leonard. leonard sheldon. sheldon no no no it is not gonna be disappointed that was so much of leonard. leonard what are you sheldon. sheldon subliminal messaging scout to be alone with my leonard. leonard w

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.6327850161632641
0.6392500382047317
trained_w2v_vanilla
penny. penny bernadette. bernadette oh you know it you know that you think it smoking in my leonard. leonard oh hey honey sorry you re welcome with the record unit penny. penny you re welcome with strict bernadette. bernadette you know cwogziwwa to be it in the sky slipper court der and then penny. penny you know that not supposed to be late in the way we could be losing and the penny. penny oh sorry sorry you ll be able to burp yourself to the dentist penny. penny well it is it penny. penny oh you re sheldon. sheldon you re welcome to the bathroom slipper of our relationship bernadette. bernadette hey guys you re losing on seniority horse and the inexplicable and you re losing sheldon. sheldon well that was in my life drool bonnet and then you do to see bernadette. bernadette you don have any idea that in the mood of the sheldon. sheldon you know what about howard. howard you know what do sheldon. sheldon oh it is simulation on

In [0]:
from os import listdir
from os.path import isfile, join
mypath = "./models"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)

['trained_vanilla_sq_7.pt', 'trained_vanilla_sq_5.pt', 'trained_GRU_sq_6.pt', 'trained_GRU_sq_5.pt', 'trained_lstm_sq_7.pt', 'trained_vanilla_sq_6.pt', 'trained_GRU_sq_3.pt', 'trained_GRU_sq_7.pt', 'trained_lstm_sq_6.pt', 'trained_lstm_sq_3.pt', 'trained_vanilla_sq_3.pt', 'trained_lstm_sq_5.pt']
